In [3]:
from typing import List
import whisper

model = whisper.load_model("tiny")

/Users/lawrencechen/fun/autochapter/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
COHERE_API_KEY = "8rLIrm60Gf9mSmeIF2RHxv1TNcjXpHFDZ9XnqjD2"

In [5]:
def timestamps2summaries(video_url: str, timestamps: List[int], output_dir: str):
  transcribed = model.transcribe(video_url)
  print(transcribed)
  # transcribed["timestamps"]

In [8]:
import os
import json

# video_url = "videos/cs61a_lec1.mkv"
# video_url = "videos/cs61b_lec1.webm"
# result = model.transcribe(video_url)
def transcribe_video(video_url: str):
  video_transcribed_path = os.path.join("transcribed", video_url.split("/")[-1].split(".")[0] + ".json")
  print(video_transcribed_path)
  if os.path.exists(video_transcribed_path):
    with open(video_transcribed_path) as f:
      return json.load(f)
  else:
    result = model.transcribe(video_url)
    with open(video_transcribed_path, "w") as f:
      json.dump(result, f)
    return result

In [10]:
transcribed = transcribe_video("videos/cs61a_lec1.mkv")

transcribed/cs61a_lec1.json


In [18]:
import cohere
co = cohere.Client(COHERE_API_KEY)

def generate_summary(passage: str):
  response = co.generate(
    model='large',
    prompt=f'Passage: Is Wordle getting tougher to solve? Players seem to be convinced that the game has gotten harder in recent weeks ever since The New York Times bought it from developer Josh Wardle in late January. The Times has come forward and shared that this likely isn’t the case. That said, the NYT did mess with the back end code a bit, removing some offensive and sexual language, as well as some obscure words There is a viral thread claiming that a confirmation bias was at play. One Twitter user went so far as to claim the game has gone to “the dusty section of the dictionary” to find its latest words.\n\nTLDR: Wordle has not gotten more difficult to solve.\n--\nPassage: ArtificialIvan, a seven-year-old, London-based payment and expense management software company, has raised $190 million in Series C funding led by ARG Global, with participation from D9 Capital Group and Boulder Capital. Earlier backers also joined the round, including Hilton Group, Roxanne Capital, Paved Roads Ventures, Brook Partners, and Plato Capital.\n\nTLDR: ArtificialIvan has raised $190 million in Series C funding.\n--\nPassage: {passage}\n\nTLDR:',
    max_tokens=50,
    temperature=0.8,
    k=0,
    p=1,
    frequency_penalty=0,
    presence_penalty=0,
    stop_sequences=["--"],
    return_likelihoods='NONE')
  # print('Prediction: {}'.format(response.generations[0].text))
  return response.generations[0].text

In [21]:
generate_summary(transcribed["text"][0:2000])

' Rachael Bale is going to give you a set of lecture about software engineering.\n--'

In [2]:
def get_segment(result, start: int, duration: int):
  assert type(result["text"]) is str, 'result["text"] is invalid'
  assert result["segments"] is not None, 'result["segments"] is invalid'
  ts = 0
  for segment in result["segments"]:
    if ts + segment["duration"] > start:
      break
    ts += segment["duration"]

  # for segment in result["segments"]:
  #   if segment["start"]
  return result["text"][start:start+duration]
get_segment(0, 60)

NameError: name 'result' is not defined